In [1]:
from src.exp_logger import logger

import pyterrier as pt  # type: ignore
import json
from src.load_index import setup_system
from tqdm import tqdm
import pandas as pd


logger.setLevel("INFO")

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
index, topics, qrels = setup_system("WT")

Loaded index with  1570734 documents.


In [3]:
id_no = {}
meta = index.getMetaIndex()

while True:
    try:
        id_no[len(id_no)] = meta.getItem("docno", len(id_no))
    except:
        print("Done")
        break

no_id = {v: k for k, v in id_no.items()}


def find_max_list(list):
    list_len = [len(i) for i in list]
    return max(list_len)

Done


In [4]:
slide = pt.text.sliding(length=150, stride=75, prepend_attr=None, text_attr="text")

def relevant_passages():
    rels = topics.merge(qrels[qrels["label"]>=1], on="qid")

    for _, query in tqdm(topics.iterrows(), total=len(topics)):
        qid = query["qid"]
        query = query["query"]

        ##### Get relevant passages #####
        rel_docs = rels[rels["qid"] == qid]["docno"].tolist()  # get relevant docs
        if not rel_docs:
            logger.warning(f"Skipping {qid}: `{query}`, not relevant docs found")
            continue


        for docno in rel_docs:
            docid = no_id[docno]
            doc = index.getMetaIndex().getItem("text", docid)
            cleaned_doc = doc.replace(docno, " ").strip()
            df = pd.DataFrame([{"text": cleaned_doc, "docno": docno}])
            passages = slide.transform(df)["text"].tolist()

            logger.info(f"Found {len(passages)} passages for {docno}")
        
            with open("../data/passages/t5/WT-relevant-passages-arb.jsonl", "a+") as f:
                for passage in passages:
                    json.dump({"qid": qid, "docno": docno, "passage": passage}, f)
                    f.write("\n")

In [5]:
def not_relevant_passages():
    bm25 = pt.BatchRetrieve(index, wmodel="BM25")
    for _, query in tqdm(topics.iterrows(), total=len(topics)):
        qid = query["qid"]
        base = bm25(topics[topics["qid"]==qid])  # baseline
        gradet = base.merge(qrels, on=["qid", "docno"])  # add grading    
        not_relevant_docs = gradet[gradet["label"]==0]["docno"].tolist()  # get not rel docs
        logger.info(f"Found {len(not_relevant_docs)} graded and not relevant docs for {qid}")

        for docno in not_relevant_docs:
            docid = no_id[docno]
            doc = index.getMetaIndex().getItem("text", docid)
            cleaned_doc = doc.replace(docno, " ").strip()

            df = pd.DataFrame([{"text": cleaned_doc, "docno": docno}])
            passages = slide.transform(df)["text"].tolist()

            logger.info(f"Found {len(passages)} passages for {docno}")
        
            with open("../data/passages/t5/WT-not-relevant-passages-arb.jsonl", "a+") as f:
                for passage in passages:
                    json.dump({"qid": qid, "docno": docno, "passage": passage}, f)
                    f.write("\n")

In [6]:
logger.setLevel("WARNING")

relevant_passages()

not_relevant_passages()

  3%|▎         | 18/672 [00:00<00:06, 94.47it/s]

calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling 

[2023-05-08 18:07:12][WARNING] src.exp_logger: Skipping q062216451: `chateau cathare`, not relevant docs found
[2023-05-08 18:07:12][WARNING] src.exp_logger: Skipping q0622724: `convert video to mp3`, not relevant docs found
  7%|▋         | 44/672 [00:00<00:05, 109.73it/s]

calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling 

  8%|▊         | 56/672 [00:00<00:05, 109.61it/s]

...

100%|█████████▉| 670/672 [09:49<00:01,  1.17it/s]

calling sliding on df of 1 rows
calling sliding on df of 1 rows


100%|█████████▉| 671/672 [09:50<00:00,  1.22it/s]

calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows
calling sliding on df of 1 rows


100%|██████████| 672/672 [09:50<00:00,  1.14it/s]
